In this notebook was described how to get haplotypes of specific regions for all patients and time periods

#### Programs to need

- bwa
- samtools
- CliqueSNV

#### Data

<a> "https://hiv.biozentrum.unibas.ch/data/" Patient Data</a>

In [10]:
from loader import Loader
import json
import pandas as pd

#### Convert all .bam to .fastq

In [ ]:
#Load .bam files for all patients
loader = Loader()
loader.load_bam()

In [9]:
#All files must be in the same folder
# path to folder with .bam files
!cd PATH
# create list of all .bam files in folder
!ls *.bam > BAM.list

In [10]:
# run samtools to convert all .bam files to .fastq
!paste BAM.list | while read BAM ; do samtools bam2fq -F 0x900 "${BAM}" > "${BAM}".fastq; done;

/bin/sh: /Users/lolitiy/Documents/inst_bioinf_19_20/project2/p1/p1_1//Users/lolitiy/Documents/inst_bioinf_19_20/project2/p1/p1_1/reads_p1_1_F4.bam.fastq: No such file or directory
/bin/sh: /Users/lolitiy/Documents/inst_bioinf_19_20/project2/p1/p1_1//Users/lolitiy/Documents/inst_bioinf_19_20/project2/p1/p1_1/reads_p1_1_F5.bam.fastq: No such file or directory


In order to reconstruct haplotypes for the desired gene, it is necessary to obtain fastq, including this region.

Here we are reconstructing haplotypes for the gp120 gene. It is located in F4 - F5 regions. Therefore, merge fastq files for F4-F5 region together.

####  Merge .fastq

In [ ]:
# Merge F4 and F5 region together for each patient
!for p in {1..11}; do for i in {1..11}; do a="reads_p${p}_${i}_F4.bam.fastq"; b="reads_p${p}_${i}_F4.bam.fastq"; c="reads_p${p}_${i}_F4_F5.bam.fastq" ; cat $a $b > $c; done ; done

#### Align reads to reference

In [ ]:
#load all references in .json
loader.load_reference()

In [ ]:
#extract from json region

In [27]:
reference_path = '/data/'
reference_list = [x for x in os.listdir(reference_path) if '.json' in x]
reference_list

['hivevo_reference_p1.json',
 'hivevo_reference_p6.json',
 'hivevo_reference_p7.json',
 'hivevo_reference_p8.json',
 'hivevo_reference_p4.json',
 'hivevo_reference_p5.json',
 'hivevo_reference_p9.json',
 'hivevo_reference_p2.json',
 'hivevo_reference_p10.json',
 'hivevo_reference_p11.json',
 'hivevo_reference_p3.json']

In [18]:
reference_df = pd.DataFrame()
for name in reference_list:
    with open(os.path.join(reference_path, name)) as f:
        json_file = json.load(f)
        # drop unnecessary columns
        t = pd.DataFrame(data=json_file).drop(['name', 'description'], axis=1)

        # divide the combined columns into separate
        t = pd.concat([t.drop(['features'], axis=1), t.features.apply(pd.Series)], axis=1)

        # translate into a simple list
        t.location = t.location.apply(pd.Series)

        # cut the sequence
        t['region_seq'] = t.apply(lambda x: x.seq[x.location[0]:x.location[1]].strip(), axis=1)

        # rename
        t.rename(mapper={'region_seq': 'sequence', 'seq': 'full_reference'}, axis=1, inplace=True)
        reference_df = pd.concat([reference_df, t], ignore_index=True)
#leave rows only with the gp120 gene       
reference_df = reference_df.query("name == 'gp120'")

In [19]:
reference_df

,full_reference,framestart,len,id,type,name,location,sequence
35,TTAAAGTGGTGTGTGCCCGTCTGTGATAGGACTCTGGTAACTAGAG...,261,9060,reference_p1,protein,gp120,"[5778, 7215]",TCAAACAACTTGTGGGTTACAGTTTATTATGGGGTTCCTGTGTGGA...
75,TCTAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAG...,244,9031,reference_p6,protein,gp120,"[5765, 7175]",ATGGGGAACTTATGGGTCACAGTCTATTATGGGGTACCTGTGTGGA...
115,TTCAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAG...,242,9104,reference_p7,protein,gp120,"[5796, 7236]",AAGGAAAAATTGTGGGTCACAGTCTATTATGGGGTACCTGTGTGGA...
155,TTCAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAG...,241,9120,reference_p8,protein,gp120,"[5771, 7271]",GCAGAAAATTTGTGGGTCACAGTCTATTATGGGGTACCTGTGTGGA...
195,TTCAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAG...,242,9059,reference_p4,protein,gp120,"[5763, 7227]",ACAAAACCATTGTGGGTTACAGTCTATTATGGGGTACCTGTGTGGA...
235,TTCAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAG...,242,8991,reference_p5,protein,gp120,"[5757, 7167]",AAGGAACAAACGTGGGTCACAGTCTATTATGGGGTACCTGTGTGGA...
275,TTCAAGTAGTGTGTGCCCATCTGTTGTGTGACTCTGGTAGCTAGAG...,243,9017,reference_p9,protein,gp120,"[5752, 7183]",ACAGGAAACTTGTGGGTCACAGTCTATTATGGGGTGCCTGTGTGGA...
315,TTTAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAG...,241,9095,reference_p2,protein,gp120,"[5768, 7229]",GGGGAACAGTTATGGGTCACAGTCTATTATGGGGTACCTGTGTGGA...
355,TTCAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAG...,243,9074,reference_p10,protein,gp120,"[5782, 7240]",GCAGAAGATAAGTGGGTCACAGTCTATTATGGGGTACCTGTGTGGA...
395,TTCAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAG...,240,9011,reference_p11,protein,gp120,"[5761, 7162]",GCAGAACAATTGTGGGTCACAGTCTATTATGGGGTACCTGTGTGGA...


In [21]:
# drop unnecessary columns
reference_df = reference_df.drop(['full_reference', 'framestart', 'len', 'type', 'name', 'location'], axis=1)

In [23]:
ref_dict = reference_df.set_index('id').T.to_dict('sequence')

In [ ]:
#To separate fasta files

In [24]:
#here you need the reference file in .fasta
#indexing reference
!for p in {1..11}; do a="hivevo_reference_p${p}_gp120.fasta"; bwa index $a; done

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index /Users/lolitiy/Documents/inst_bioinf_19_20/project2/patients/p2_74d/genome_p2.fasta
[main] Real time: 0.007 sec; CPU: 0.013 sec


In [11]:
#Align reads to reference
!for p in {1..11}; do for i in {1..11}; do a="hivevo_reference_p${p}_gp120.fasta"; b="reads_p${p}_${i}_F4_F5.bam.fastq"; c="reads_sorted_p${p}_${i}_F4_F5.bam"; bwa mem $a $b | samtools view -S -b - | samtools sort - -o $c ; done ; done

[E::bwa_idx_load_from_disk] fail to locate the index files


In [38]:
#indexing .bam
!for p in {1..11}; do for i in {1..11}; do a="reads_sorted_p${p}_${i}_F4_F5.bam" ; samtools index $a ; done ; done

In [57]:
#.bam convert to .sam (if need header, add -h)
!for p in {1..11}; do for i in {1..11}; do a="reads_sorted_p${p}_${i}_F4_F5.bam" ; b="reads_sorted_p${p}_${i}_F4_F5.sam" ; samtools view -h $a > $b ; done ; done

#### Haplotype reconstruction for custom region

In [58]:
#reconstruct haplotypes with CliqueSNV
!for p in {1..11}; do for i in {1..11}; do a="reads_sorted_p${p}_${i}_F4_F5.sam" ; java -jar clique-snv.jar -m snv-illumina -in $a -outDir p${p}_${i} -tf 0.001 -t 10 ; done ; done

CliqueSNV version: 1.5.0
Settings: {-m=snv-illumina, -in=/Users/lolitiy/Documents/inst_bioinf_19_20/project2/patients/p2_74d/p2_74_V3.sam, -outDir=Users/lolitiy/Documents/inst_bioinf_19_20/project2/patients/p2_74d, -t=10, -tf=0.001}
Start read sam
0 DONE
Start convert
 DONE
Total reads number: 70625
read 5326
Start iterations EM
58
Start build clusters
 - DONE
Start EM 
Start iterations EM
0
 - DONE 
[]
Found haplotypes 1
Haplotypes after 0.1 1
Start build clusters
 - DONE
Start EM 
Start iterations EM
0
 - DONE 
[]
Found haplotypes 1
Haplotypes after 0.05 1
Start build clusters

 - DONE
Start EM 
Start iterations EM
70
 - DONE 
[]
A[6523]
Found haplotypes 2
Haplotypes after 0.01 1
Haplotypes after no freq limit 6

Freq before filtering[0.9762637414665623, 0.021077090682982726, 0.0013548670263076972, 4.8806490992950535E-4, 4.591855645212491E-4, 3.570503496965231E-4]
Haplotypes before return 3
Results are available in: /Users/lolitiy/Users/lolitiy/Documents/inst_bioinf_19_20/project2/pa